In [2]:
import requests, pickle
from pathlib import Path

token = '?token=0GRsvLM7WAKI_IYq4TTVxQ44oa4PNCz0P1dLKISoDnsTSpG8qAk'

# Static champion data
champions = 'https://api.pandascore.co/lol/v1/champions'

# Base URLs
allTournamentsURL = 'http://api.pandascore.co/lol/v1/tournamentlist'
baseMatchURL = 'https://api.pandascore.co/lol/v1/matches/'
baseTournamentURL = 'https://api.pandascore.co/lol/v1/tournaments/'
basePlayerURL = 'http://api.pandascore.co/lol/v1/players/'
baseChampionsURL = 'http://api.pandascore.co/lol/v1/champions'

In [3]:
champions = dict()

# get list of champions
if(Path('champions.pickle').is_file()): #check if data already exists
    with open('champions.pickle', 'rb') as f:
        champions = pickle.load(f)
else:
    r_champions = requests.get(baseChampionsURL+token)
    champs = r_champions.json()
    #map champID -> champion for easier access
    smallID = 0
    for champ in champs:
        champ['smallID'] = smallID
        champions[champ['id']] = champ
        smallID += 1
    with open('champions.pickle', 'wb') as f:
        pickle.dump(champions, f)
        
championNames_dict = dict()
if(Path('championNames_dict.pickle').is_file()):
    with open('championNames_dict.pickle', 'rb') as f:
        championNames_dict = pickle.load(f)
else:
    for key, champ in champions.items():
        championNames_dict[champ['name']] = champ
    with open('championNames_dict.pickle', 'wb') as f:
        pickle.dump(championNames_dict, f)

In [4]:
tournaments = list()

# get list of tournaments
if(Path('tournaments.pickle').is_file()):
    with open('tournaments.pickle', 'rb') as f:
        tournaments = pickle.load(f)
else:
    r_tournaments = requests.get(allTournamentsURL + token)
    tournaments = r_tournaments.json()
    #pickle tournaments
    with open('tournaments.pickle', 'wb') as f:
        pickle.dump(tournaments, f)

print(len(tournaments))

84


In [5]:
matchIDs = set()

if(Path('matchIDs.pickle').is_file()):
    with open('matchIDs.pickle', 'rb') as f:
        matchIDs = pickle.load(f)
else:
    # get list of match IDs from tournaments
    for t in tournaments:
        r_tournament = requests.get(baseTournamentURL + str(t['id']) + token)
        tournament = r_tournament.json()
        for m in tournament['matches']:
            matchIDs.add(m['id'])
        print(tournament['name'] + ': (' + str(tournament['id']) + '): ' + str(len(matchIDs)))

    #pickle matchIDs
    with open('matchIDs.pickle', 'wb') as f:
        pickle.dump(matchIDs, f)

print(len(matchIDs))

1534


In [6]:
matchData = list()

if(Path('matchData.pickle').is_file()):
    with open('matchData.pickle', 'rb') as f:
        matchData = pickle.load(f)
else:
    for i, matchID in enumerate(matchIDs):
        r_match = requests.get(baseMatchURL + str(matchID) + token)
        match = r_match.json()
        matchData.append(match)
        
        print(str(i) + ') ' + str(match['id']) + ': ' + match['name'])

    # store the object data into a pickle
    with open('matchData.pickle', 'wb') as f:
        pickle.dump(matchData, f)

print(len(matchData))

# TODO:
# only store the champion, ban, player, teamID, and winner data in a separate object
# filter out matches that don't have team/roster/winner/champion information or aren't on Summoner's rift or aren't 5v5


1534


In [7]:
playerIDs = set()

# get set of unique playerIDs
for match in matchData:
    if not match['teams'] is None:
        for team in match['teams']:
            if 'roster' in team:
                for player in team['roster']:
                    if type(player) is dict:
                        playerIDs.add(player['id'])

print(len(playerIDs))

players = dict()

if(Path('players_dict.pickle').is_file()):
    with open('players.pickle', 'rb') as f:
        players = pickle.load(f)
else:
    for i, playerID in enumerate(playerIDs):
        r_player = requests.get(basePlayerURL + str(playerID) + token)
        player = r_player.json()
        if not player['name'] is None:
            print(str(i) + ') ' + str(playerID) + ': ' + player['name'])
        players[playerID] = player
        
    # store the player data into a pickle
    with open('players_dict.pickle', 'wb') as f:
        pickle.dump(players, f)

print(len(players))

669
669


# Game Schema
ID //game ID
tip_<position>_champID //champID for all players on all teams.
ti_avg_AD //avg attack damage for each team
ti_avg_HP //avg HP for each team
ti_avg_AS //avg attack speed for each team
ti_avg_MR //avg magic resist for each team
ti_avg_ARM //avg armor for each team
winner //0 if t0 won; 1 if t1 won

NOTES:
-all avg stats are taken at level 11 (good or bad?? favors good mid game champions)

FUTURE:
-tipi_skill 
//skill rating for each player (if player doesn't have a skill, the avg skill of their team is used)
//unavailable because API doesn't return player skill

-ti_color
//not sure if useful
//data may be null often

-ti_ID //team ID. Not really needed other than identification infomation

In [39]:
gameData = list()

winCount = dict()
gameCount = dict()

def getTeamNumber(m, teamID):
    return 0 if teamID == m['t0_ID'] else 1 if teamID == m['t1_ID'] else -1

class InputError(Exception):
    def __init__(self, expression, message):
        self.expression = expression #input expression in which the error occurred
        self.message = message #explanation of the error

championStats = ['armorperlevel','mpperlevel','attackdamage','hpregen','critperlevel','hpregenperlevel','spellblockperlevel',
                 'attackdamageperlevel','hp','attackspeedperlevel','movespeed','hpperlevel','crit','mpregen','mpregenperlevel',
                 'attackrange','spellblock','mp','armor','attackspeedoffset']

if(Path('gameData.pickle').is_file()):
    with open('gameData.pickle', 'rb') as f:
        pass
else:
    # get the winpct for each team
    for match in matchData:
        if match is None or len(match['games']) < 0:
            continue
        
        if not match['games'] is None:
            for game in match['games']:
                try:
                    if len(game['game_teams']) != 2:
                        raise InputError('Wrong number of teams', str(len(game['game_teams'])))    
                    if game['winner_id'] != game['game_teams'][0]['team_id'] and game['winner_id'] != game['game_teams'][1]['team_id']:
                        continue
                    for team in game['game_teams']:
                        gameCount[team['team_id']] = 1 if team['team_id'] not in gameCount else gameCount[team['team_id']]+1
                    winCount[game['winner_id']] = 1 if game['winner_id'] not in winCount else winCount[team['team_id']]+1
                except (TypeError, KeyError, InputError):
                    continue

if(Path('gameData.pickle').is_file()):
    with open('gameData.pickle', 'rb') as f:
        gameData = pickle.load(f)   
else:
    # clean the match json into a list of simple dicts for each game, based on schema
    for match in matchData:
        if match is None or len(match['games']) < 0:
            continue
        
        # match data (doesn't change between games. eg. team id, roster)
        m = dict()
        try:
            if len(match['teams']) != 2:
                raise InputError('Wrong number of teams', str(len(match['teams'])))
            for i, team in enumerate(match['teams']):
                m['t'+str(i)+'_ID'] = team['id']
        except (TypeError, KeyError, InputError):
            continue
        
        if not match['games'] is None:
            for game in match['games']:
                try:
                    # building data for each game
                    g = dict()
                    if len(game['game_teams']) != 2:
                        raise InputError('Wrong number of teams', str(len(game['game_teams'])))
                    for team in game['game_teams']:
                        teamNum = str(getTeamNumber(m, team['team_id']))
                        if(teamNum == '-1'):
                            raise InputError('Team not found', str(team['team_id']) + str(m))

                        g['t'+teamNum+'_winPct'] = winCount[team['team_id']]/gameCount[team['team_id']]

                        # initialize all the stats to 0                        
                        for stat in championStats:
                            g['t'+teamNum+'_'+stat] = 0
                        
                        for player in team['game_player_teams']:
                            champID = player['champion_id']
                            g['t'+teamNum+'p_'+player['position']+'_champID'] = champID
                            champ = champions[champID]
                                                        
                            for stat in championStats:
                                g['t'+teamNum+'_'+stat] += champ[stat]
                                

                    g['ID'] = game['id']
                    g['winner'] = getTeamNumber(m, game['winner_id'])
                    gameData.append(g)
                except (TypeError, KeyError, InputError):
                    continue
                
        
    with open('gameData.pickle', 'wb') as f:
        pickle.dump(gameData, f)

In [9]:
# get more useful aggregated champion stats from champions.gg
import json

with open('championWinRates.json') as f:
    championStats = json.load(f)

# initialize all WRs to 50 (%) which is moved down to 20 (%)
for k, champ in champions.items():
    for role in ['Jungle', 'Top', 'ADC', 'Support', 'Middle']:
        champ['WR_'+role] = 20
    
for stat in championStats:
    champName = stat['title']
    role = stat['role']
    winrate = stat['general']['winPercent'] - 30
    # NOTE: can use more champion stats if needed
    
    champions[championNames_dict[champName]['id']]['WR_' + role] = winrate

with open('champions.pickle', 'wb') as f:
    pickle.dump(champions, f)

In [10]:
roles = set()
for stat in championStats:
    roles.add(stat['role'])
    
print(roles)

{'Top', 'ADC', 'Support', 'Middle', 'Jungle'}
